Universidade Federal do Rio Grande do Sul (UFRGS) <br>
Programa de Pós-Graduação em Engenharia Civil (PPGEC) <br>
PEC00100: Tópicos de Problemas de Interação Solo-Estrutura <br>

_Daniele Kauctz Monteiro_<br>
[(GitHub)](https://github.com/danielekauctz) | <a href="mailto:danielekauctz@hotmail.com">[(E-mail)]</a>
________________________________________________________________________________________________________________________________

# Análise de um Túnel no Estado Plano de Deformações (EPD)

[0.   Inicialização do Jupyter notebook](#section_0)  
[1.   Definição da solução analítica em Elasticidade e Plasticidade](#section_1)  
[2.   Análise do túnel ](#section_2) <br> 
   [2.1. Solução Elástica](#section_21) <br>
   [2.2. Solução Elasto-plástica](#section_22) <br>
[3.   Solução numérica](#section_3)  
   [3.1. Comparação das soluções analíticas e numéricas em Elasticidade](#section_31)  
[4.   Referências](#section_4) 

## 0. Inicialização do Jupyter notebook <a name="section_0"></a>
Nesse Jupyter notebook, são utilizados os seguintes repositórios e configurações de plotagem:

In [ ]:
# Repositórios
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import collections
import warnings
warnings.filterwarnings("ignore")
import matplotlib

# Plotagem
from matplotlib import rc
rc('font',**{'family':'Times New Roman','size' :12})
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rc('text', usetex=True)
matplotlib.rc('axes', labelsize=12)
matplotlib.rc('xtick', labelsize=12) 
matplotlib.rc('ytick', labelsize=12)
matplotlib.rc('lines', lw=1.0,color='k')
matplotlib.rc('axes',lw=0.75)
matplotlib.rc('legend', fontsize=10)

## 1. Definição da solução analítica em Elasticidade e Plasticidade <a name="section_1"></a> 

A solução analítica em elasticidade (deslocamento radial $u$ e tensões $\sigma_{rr}$, $\sigma_{\theta\theta}$ e $\sigma_{zz}$) de um túnel no estado plano de deformações é definifida a partir das seguintes propriedades: raio interno ($R_i$), raio externo ($R_e$), módulo de elasticidade ($E$), coeficiente de Poison ($\nu$), coesão ($c$), pressão do maciço ($P_\infty$), pressão  interna ($P_i$) e raio ($r$). Sendo assim:

In [ ]:
def elastic_solution(Ri, Re, E, nu, c, Pinf, Pi, r):

    u_e      = lambda r: (-(1+nu)/E *(Pinf-Pi)*(Ri/r)**2)*r    # deslocamento em no eixo z
    sigma_rr_e = lambda r: (Pinf-Pi)*(Ri/r)**2 - Pinf          # componente de tensão no eixo r
    sigma_tt_e = lambda r: -(Pinf-Pi)*(Ri/r)**2 - Pinf         # componente de tensão no eixo teta
    sigma_zz_e = lambda r: -2*nu*Pinf + (r*0)                  # componente de tensão no eixo z

    u       = u_e(r)
    sigma_r = sigma_rr_e(r)
    sigma_t = sigma_tt_e(r)
    sigma_z = sigma_zz_e(r)

    return u, sigma_r, sigma_t, sigma_z

Já a solução analítica em elasto-plasticidade pode ser dada pela:
- solução elástica, se $0 \geq y \geq R_i$; 
- solução elasto-plástica, se $R_e \geq y \geq R_i$. <br>

Isso segue a seguinte formulação:

In [ ]:
def elastoplastic_solution(Ri, Re, E, nu, c, Pinf, Pi, r):

    y = Ri*np.exp((Pinf-Pi)/(2*c) - (1/2))        # raio plástico

    # Solução Elástica
    u_e      = lambda r: (-(1+nu)/E *(Pinf-Pi)*(Ri/r)**2)*r
    sigma_rr_e = lambda r: (Pinf-Pi)*(Ri/r)**2 - Pinf
    sigma_tt_e = lambda r: -(Pinf-Pi)*(Ri/r)**2 - Pinf
    sigma_zz_e = lambda r: -2*nu*Pinf + (r*0)

    # Solução Elasto-Plástica
    u_p1        = lambda r: ((1+nu)/E*( (-2*(1-nu)*c*(y/r)**2) + ((1-2*nu)*(Pinf-Pi-(2*c*np.log(r/Ri)))) ))*r
    sigma_rr_p1 = lambda r: (-2*c*np.log(r/Ri)) - Pi
    sigma_tt_p1 = lambda r: (-2*c*(1+np.log(r/Ri))) - Pi  
    sigma_zz_p1 = lambda r: 2*nu*(Pi - (2*c*(np.log(r/Ri)+1/2)) )

    u_p2        = lambda r: (-(1+nu)/E*c*(y/r)**2)*r
    sigma_rr_p2 = lambda r: c*(y/r)**2 - Pinf   
    sigma_tt_p2 = lambda r: -c*(y/r)**2 - Pinf
    sigma_zz_p2 = lambda r: -2*nu*Pinf

    # Deslocamentos e tensões
    if 0 <= y <=Ri:                         # solução elástica
        u       = u_e(r)
        sigma_r = sigma_rr_e(r)
        sigma_t = sigma_tt_e(r)
        sigma_z = sigma_zz_e(r)

    elif Ri <= y <= Re:                     # solução elasto-plástica
        if isinstance(r, (collections.Sequence, np.ndarray)):
            u       = np.zeros(len(r))
            sigma_r = np.zeros(len(r))
            sigma_t = np.zeros(len(r))
            sigma_z = np.zeros(len(r))

            for i in range(len(r)):
                if r[i] <= y:
                    u[i]       = u_p1(r[i])
                    sigma_r[i] = sigma_rr_p1(r[i])
                    sigma_t[i] = sigma_tt_p1(r[i])
                    sigma_z[i] = sigma_zz_p1(r[i])
                elif r[i] > y:
                    u[i]       = u_p2(r[i])
                    sigma_r[i] = sigma_rr_p2(r[i])
                    sigma_t[i] = sigma_tt_p2(r[i])
                    sigma_z[i] = sigma_zz_p2(r[i])
        else:
            if r <= y:
                u       = u_p1(r)
                sigma_r = sigma_rr_p1(r)
                sigma_t = sigma_tt_p1(r)
                sigma_z = sigma_zz_p1(r)
            elif r > y:
                u       = u_p2(r[i])
                sigma_r = sigma_rr_p2(r)
                sigma_t = sigma_tt_p2(r)
                sigma_z = sigma_zz_p2(r)
                    
    return u, sigma_r, sigma_t, sigma_z

Já a curva de convergência é dada por uma parcela inicial elástica (quando $P_i > P_\infty - c$) e por uma parcela plástica (quando $P_i \leq P_\infty - c$), de acordo com:

In [ ]:
def convergence_curve(E, nu, c, Pinf, Pi):
    
    Plim = Pinf - c
    
    Ui = np.zeros(len(Pi))
    for j in range(len(Pi)):
        if Pi[j] > Plim:
            Ui[j] = ((1 + nu)/E *(Pinf - Pi[j]))
    
        elif Pi[j] <= Plim: 
            Ui[j] = (1 + nu)/E * ((2*(1-nu)*c*np.exp((Pinf - Pi[j] - c)/c)) - ((1 - 2*nu)*(Pinf-Pi[j])))
    
    return Ui

Destaca-se que a curva de convergência não varia com a mudança do $R_i$, sendo ela dependente apenas das propriedades do maciço rochoso.

## 2. Análise do túnel <a name="section_2"></a> 

Com as funções definidas anteriormente, um túnel com as seguintes propriedades é analisado:

In [ ]:
Ri = 5            # raio interno [m]
Re = Ri*20        # raio externo [m]
E  = 403          # módulo de elasticidade [MPa]
nu = 0.39         # coefiente de Poison [adimensional]
c  = 2.7          # desviador/coesão [MPa]

Pinf = 8          # pressão de confinamento do maciço [MPa]
Pi   = 0          # pressão interna [MPa]

n = 100                        # nº de pontos analisados na solução analítica
r = np.linspace(Ri, Re, n)     # variação do raio de acordo com os parâmetros do maciço

Vale ressaltar que os parâmetros do maciço rochoso foram extraídos de Quevedo (2021, p.146) para um maciço argiloso dúctil.

### 2.1. Solução Elástica <a name="section_21"></a> 

Tem-se como solução elástica:

In [ ]:
u, sigma_r, sigma_t, sigma_z = elastic_solution(Ri, Re, E, nu, c, Pinf, Pi, r)

# Plotagem dos deslocamentos
plt.figure(1,figsize=(9,5))
plt.plot(r,u, color='#0075c4')
plt.grid(True)
plt.xlim([r[0],r[-1]])
plt.xlabel('r (m)')
plt.ylabel('u (m)')
plt.title('Deslocamentos')
plt.savefig('figures/fig1.svg')

# Plotagem das tensões
plt.figure(2,figsize=(9,5))
plt.plot(r,sigma_r, label=r'$\sigma_{rr}$', color='#0075c4')
plt.plot(r,sigma_t, label=r'$\sigma_{\theta\theta}$', color='#c4b100')
plt.plot(r,sigma_z, label=r'$\sigma_{zz}$', color='#c40013')
plt.grid(True)
plt.xlim([r[0],r[-1]])
plt.xlabel('r (m)')
plt.ylabel(r'$\sigma_{ij}$ (MPa)')
plt.legend(loc = 'upper right')
plt.title('Tensões')
plt.savefig('figures/fig2.svg')

# Plotagem da Curva de Convergência
plt.figure(3,figsize=(9,5))
P = np.linspace(Pinf, 0, n)
U = convergence_curve(E, nu, c, Pinf, P)
plt.plot(U,P, color='#0075c4')
plt.grid(True)
plt.xlim([U[0],0.01])
plt.ylim([Pinf-c,P[0]])
plt.xlabel(r'$U(r_i)/r_i$')
plt.ylabel(r'$P_i$ (MPa)')
plt.title('Curva de Convergência');
plt.savefig('figures/fig3.svg')

<img src="figures/fig1.svg" alt="Fig1" width="800px"/> 
<br>
<img src="figures/fig2.svg" alt="Fig2" width="800px"/> 
<br>
<img src="figures/fig3.svg" alt="Fig3" width="800px"/> 

### 2.2. Solução Elasto-plástica <a name="section_22"></a> 

Já a solução elasto-plástica é:

In [ ]:
u_ep, sigma_r_ep, sigma_t_ep, sigma_z_ep = elastoplastic_solution(Ri, Re, E, nu, c, Pinf, Pi, r)

# Plotagem dos deslocamentos
plt.figure(4,figsize=(9,5))
plt.plot(r,u_ep, color='#0075c4')
plt.grid(True)
plt.xlim([r[0],r[-1]])
plt.xlabel('r (m)')
plt.ylabel('u (m)')
plt.title('Deslocamentos')
plt.savefig('figures/fig4.svg')

# Plotagem das tensões
plt.figure(5,figsize=(9,5))
plt.plot(r,sigma_r_ep, label=r'$\sigma_{rr}$', color='#0075c4')
plt.plot(r,sigma_t_ep, label=r'$\sigma_{\theta\theta}$', color='#c4b100')
plt.plot(r,sigma_z_ep, label=r'$\sigma_{zz}$', color='#c40013')
plt.grid(True)
plt.xlim([r[0],r[-1]])
plt.xlabel('r (m)')
plt.ylabel(r'$\sigma_{ij}$ (MPa)')
plt.legend(loc = 'upper right')
plt.title('Tensões')
plt.savefig('figures/fig5.svg')

# Plotagem da Curva de Convergência
plt.figure(6,figsize=(9,5))
P = np.linspace(Pinf, 0, n)
U = convergence_curve(E, nu, c, Pinf, P)
plt.plot(U,P, color='#0075c4')
plt.axhline(y = Pinf-c, color = 'k', ls = '--')
plt.text(0.067,5.6,'$[Elasticidade]$', color = 'k', zorder = 4, horizontalalignment='center',  
         bbox = dict(boxstyle='square,pad=0.2', ec="k", lw=0, fc="white"))
plt.text(0.067,4.7,'$[Plasticidade]$', color = 'k', zorder = 4, horizontalalignment='center',  
         bbox = dict(boxstyle='square,pad=0.2', ec="k", lw=0, fc="white"))
plt.grid(True)
plt.xlim([U[0],U[-1]])
plt.xlabel(r'$U(r_i)/r_i$')
plt.ylabel(r'$P_i$ (MPa)')
plt.title('Curva de Convergência')
plt.savefig('figures/fig6.svg');

<img src="figures/fig4.svg" alt="Fig4" width="800px"/> 
<br>
<img src="figures/fig5.svg" alt="Fig5" width="800px"/> 
<br>
<img src="figures/fig6.svg" alt="Fig6" width="800px"/> 

## 3. Solução numérica <a name="section_3"></a> 

Os deslocamentos e tensões do mesmo túnel em Elasticidade também são obtidas numericamente utilizando o software ANSYS, conforme o seguinte modelo [(script APDL)](https://1drv.ms/t/s!Agfe7UnleCo1gvJC-6fsjf2_9GO2rw?e=Psn368): 
<br>
<img src="Túnel EPD - malha.svg" alt="Modelo numérico do túnel" width="750px"/> 
<br>
Abaixo são mostrados, respectivamente, os resultados de deslocamento radial e componentes de tensão $\sigma_{rr}$, $\sigma_{\theta\theta}$ e $\sigma_{zz}$.
<br>
<img src="figures/Túnel EPD - u.svg" alt="Deslocamentos" width="750px"/> 
<img src="figures/Túnel EPD - sigmarr.svg" alt="Sigma_rr" width="750px"/>
<img src="figures/Túnel EPD - sigmatt.svg" alt="Sigma_tt" width="750px"/>
<img src="figures/Túnel EPD - sigmazz.svg" alt="Sigma_zz" width="750px"/>

### 3.1. Comparação das soluções analíticas e numéricas em Elasticidade <a name="section_31"></a> 
Buscando comparar os resultados analíticos e numéricos, as soluções em Elasticidade são plotadas a seguir.

In [ ]:
data = pd.read_csv('Results_ANSYS.txt',delim_whitespace=True, header = None,dtype = 'float')

S = data[0]+Ri                  # variação do raio
u_ansys = data[1]
sigma_r_ansys = data[2]
sigma_t_ansys = data[3]
sigma_z_ansys = data[4]

# Plotagem dos deslocamentos
plt.figure(7,figsize=(9,5))
plt.plot(r,u, label='solução analítica', color='#0075c4')
plt.scatter(S,u_ansys, label='solução numérica', color='#0075c4', s=10)
plt.grid(True)
plt.xlim([r[0],r[-1]])
plt.xlabel('r (m)')
plt.ylabel('u (m)')
plt.legend(loc = 'lower right')
plt.title('Deslocamentos')
plt.savefig('figures/fig7.svg')

# Plotagem das tensões
plt.figure(8,figsize=(9,5))
plt.plot(r,sigma_r, label=r'$\sigma_{rr,a}$', color='#0075c4')
plt.scatter(S,sigma_r_ansys, label=r'$\sigma_{rr,n}$', s=10, color='#0075c4')
plt.plot(r,sigma_t, label=r'$\sigma_{\theta\theta,a}$', color='#c4b100')
plt.scatter(S,sigma_t_ansys, label=r'$\sigma_{\theta\theta,n}$', s=10, color='#c4b100')
plt.plot(r,sigma_z, label=r'$\sigma_{zz,a}$', color='#c40013')
plt.scatter(S,sigma_z_ansys, label=r'$\sigma_{zz,n}$', s=10, color='#c40013')
plt.grid(True)
plt.xlim([r[0],r[-1]])
plt.xlabel('r (m)')
plt.ylabel(r'$\sigma_{ij}$ (MPa)')
plt.legend(loc = 'upper right', ncol=3)
plt.title('Tensões')
plt.savefig('figures/fig8.svg');

<img src="figures/fig7.svg" alt="Fig7" width="800px"/> 
<br>
<img src="figures/fig8.svg" alt="Fig8" width="800px"/> 

## 4. Referências <a name="section_4"></a>
- QUEVEDO, F. P. M. <b>Análise Computacional das Deformações em Túneis Profundos considerando o Acoplamento Plasticidade-Viscoplasticidade</b>. 2021. 226p. Tese (Doutorado em Engenharia) – Programa de Pós-Graduação em Engenharia Civil, Universidade Federal do Rio Grande do Sul, Porto Alegre. 
- QUEVEDO, F. P. M.; BERNAUD, D.; MAGHOUS, S. <b>Notas de Aula - Tópicos de Problemas de Interação Solo-Estrutura</b>. 2023. Universidade Federal do Rio Grande do Sul, Porto Alegre.